In [ ]:
from collections.abc import Iterator
from random import randint, choice

N = 8  # Board size

def heuristic(board: str) -> int:
    """The heuristic cost function h is the number of pairs of queens that are attacking each other;
    this will be zero only for solutions. (It counts as an attack if two pieces are in the same line,
    even if there is an intervening piece between them.)
    """
    h = 0

    for i in range(N):
        # Value in column i
        row_i = int(board[i])

        for j in range(i + 1, N):
            # Value in column j
            row_j = int(board[j])

            # Same row
            if row_i == row_j:
                h += 1

            # Same diagonal
            if (j - i) == abs(row_j - row_i):
                h += 1

    return h

def random_state() -> str:
    res = ''

    for col in range(N):
        res += str(randint(1, N))

    return res

def successors(current: str) -> list[str]:
    """Generate all the successors of a state, i.e. all possible states generated by moving a single queen
    to another square in the same column (so each state has 8x7=56 successors)."""
    successors = []

    for col in range(N):
        current_row = int(current[col])

        for new_row in range(1, N + 1):
            if new_row != current_row:
                # Create new board with queen moved
                new_board = current[:col] + str(new_row) + current[col + 1 :]
                successors.append(new_board)

    return successors

def successors_generator(current: str) -> Iterator[str]:
    for col in range(N):
        current_row = int(current[col])

        for new_row in range(1, N + 1):
            if new_row != current_row:
                # Create new board with queen moved
                new_board = current[:col] + str(new_row) + current[col + 1 :]
                yield new_board

def display(board: str) -> None:
    grid = [['_'] * N for _ in range(N)]

    for col in range(N):
        current_row = int(board[col])
        grid[N - current_row][col] = 'X'

    for row in grid:
        print(' '.join(row))

In [2]:
def hill_climbing(initial_state: str) -> str:
    current, h_current = initial_state, heuristic(initial_state)
    steps = 0

    while True:
        children = [(successor, heuristic(successor)) for successor in successors(current)]
        neighbor, h_neighbor = min(children, key=lambda x: x[1])
        steps += 1
        if h_neighbor >= h_current:
            print(f'Steps: {steps}')
            return current
        else:
            current, h_current = neighbor, h_neighbor

def hill_climbing_sideways_move(initial_state: str) -> str:
    current, h_current = initial_state, heuristic(initial_state)
    sideways_move = 0
    steps = 0

    while True:
        children = [(successor, heuristic(successor)) for successor in successors(current)]
        h_min = min(children, key=lambda x: x[1])[1]
        children = [child for child in children if child[1] == h_min]
        # We choose a random neighbor among the best ones
        neighbor, h_neighbor = choice(children)
        steps += 1
        if (h_neighbor > h_current) or (sideways_move == 100):
            print(f'Steps: {steps}')
            return current
        elif h_neighbor == h_current:
            sideways_move += 1
            current, h_current = neighbor, h_neighbor
        else:
            current, h_current = neighbor, h_neighbor

def first_choice_hill_climbing(initial_state: str) -> str:
    current, h_current = initial_state, heuristic(initial_state)
    steps = 0

    while True:
        for child in successors_generator(current):
            h_child = heuristic(child)
            
            if h_child < h_current:
                steps += 1
                current, h_current = child, h_child
                break
        # finally found the use of an 'else' on a 'for' loop, fuck yeah!
        else:
            print(f'Steps: {steps}')
            return current

def random_restart_hill_climbing() -> str:
    current_state = random_state()
    restarts = 0

    while True:
        res = hill_climbing(current_state)
        heuristic_res = heuristic(res)
        if heuristic_res == 0:
            print(f'Restart: {restarts}')
            return res
        else:
            restarts += 1
            current_state = current_state = random_state()

def random_restart_hill_climbing_sideways_move() -> str:
    current_state = random_state()
    restarts = 0

    while True:
        res = hill_climbing_sideways_move(current_state)
        heuristic_res = heuristic(res)
        if heuristic_res == 0:
            print(f'Restart: {restarts}')
            return res
        else:
            restarts += 1
            current_state = current_state = random_state()



In [3]:
s = '43254323'
display(s)
print(s)

_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ X _ _ _ _
X _ _ _ X _ _ _
_ X _ _ _ X _ X
_ _ X _ _ _ X _
_ _ _ _ _ _ _ _
43254323


In [4]:
print(">>> Hill Climbing:")
res = hill_climbing(s)
display(res)
print(res)
print(heuristic(res))

>>> Hill Climbing:
Steps: 6
_ _ _ _ _ _ _ _
_ _ _ _ _ _ X _
_ X _ _ _ _ _ _
_ _ _ X _ _ _ _
_ _ _ _ _ X _ _
X _ _ _ _ _ _ X
_ _ X _ _ _ _ _
_ _ _ _ X _ _ _
36251473
1


In [5]:
print(">>> Hill Climbing Sideways Move:")
res = hill_climbing_sideways_move(s)
display(res)
print(res)
print(heuristic(res))

>>> Hill Climbing Sideways Move:
Steps: 41
_ _ _ _ X _ _ _
_ _ X _ _ _ _ _
X _ _ _ _ _ _ _
_ _ _ _ _ _ X _
_ X _ _ _ _ _ _
_ _ _ _ _ _ _ X
_ _ _ _ _ X _ _
_ _ _ X _ _ _ _
64718253
0


In [6]:
print(">>> First Choice Hill Climbing:")
res = first_choice_hill_climbing(s)
display(res)
print(res)
print(heuristic(res))

>>> First Choice Hill Climbing:
Steps: 11
_ X _ _ _ _ _ _
_ _ _ _ _ X _ _
X _ _ _ _ _ _ _
_ _ X _ _ _ _ _
_ _ _ X _ _ _ _
_ _ _ _ _ _ _ X
_ _ _ _ _ _ X _
_ _ _ _ X _ _ _
68541723
2


In [7]:
### From a random state

In [8]:
r = random_state()
display(r)
print(r)

_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _
_ X _ _ _ _ _ _
X _ _ _ X X _ _
_ _ _ X _ _ X _
_ _ X _ _ _ _ _
_ _ _ _ _ _ _ X
_ _ _ _ _ _ _ _
56345542


In [9]:
print(">>> Hill Climbing:")
res = hill_climbing(r)
display(res)
print(res)
print(heuristic(res))

>>> Hill Climbing:
Steps: 5
X _ _ _ _ _ X _
_ _ _ _ X _ _ _
_ X _ _ _ _ _ _
_ _ _ _ _ X _ _
_ _ _ _ _ _ _ _
_ _ X _ _ _ _ _
_ _ _ _ _ _ _ X
_ _ _ X _ _ _ _
86317582
1


In [10]:
print(">>> Hill Climbing Sideways Move:")
res = hill_climbing_sideways_move(r)
display(res)
print(res)
print(heuristic(res))

>>> Hill Climbing Sideways Move:
Steps: 7
_ _ _ _ X _ _ _
_ _ _ _ _ _ X _
_ X _ _ _ _ _ _
_ _ _ _ _ X _ _
_ _ X _ _ _ _ _
X _ _ _ _ _ _ _
_ _ _ _ _ _ _ X
_ _ _ X _ _ _ _
36418572
0


In [11]:
print(">>> First choice Hill Climbing:")
res = first_choice_hill_climbing(r)
display(res)
print(res)
print(heuristic(res))

>>> First choice Hill Climbing:
Steps: 9
_ _ _ X _ _ _ _
_ _ _ _ _ _ X _
X _ _ _ _ _ _ _
_ _ _ _ _ X _ _
_ _ _ _ _ _ _ _
_ X _ _ _ _ _ _
_ _ _ _ _ _ _ X
_ _ X _ X _ _ _
63181572
1


In [12]:
# %timeit hill_climbing(r)
# %timeit hill_climbing_sideways_move(r)
# %timeit first_choice_hill_climbing(r)

In [ ]:
### Random restart algorithms

In [13]:
print(">>> Random restart Hill Climbing:")
res = random_restart_hill_climbing()
display(res)
print(res)
print(heuristic(res))

>>> Random restart Hill Climbing:
Steps: 4
Steps: 6
Restart: 1
_ _ X _ _ _ _ _
_ _ _ _ _ X _ _
_ X _ _ _ _ _ _
_ _ _ _ _ _ X _
_ _ _ _ X _ _ _
X _ _ _ _ _ _ _
_ _ _ _ _ _ _ X
_ _ _ X _ _ _ _
36814752
0


In [14]:
print(">>> Random restart Hill Climbing Sideways Move:")
res = random_restart_hill_climbing_sideways_move()
display(res)
print(res)
print(heuristic(res))

>>> Random restart Hill Climbing Sideways Move:
Steps: 21
Restart: 0
_ _ X _ _ _ _ _
_ _ _ _ _ _ X _
_ X _ _ _ _ _ _
_ _ _ _ _ _ _ X
_ _ _ _ X _ _ _
X _ _ _ _ _ _ _
_ _ _ X _ _ _ _
_ _ _ _ _ X _ _
36824175
0


In [15]:
# res = set()
# while len(res) < 92:
#     res.add(random_restart_hill_climbing())
# len(res)
# # res

In [16]:
# res = set()
# while len(res) < 92:
#     res.add(random_restart_hill_climbing_sideways_move())
# len(res)
# # res